In [2]:
import cv2, os, pyautogui, time
import matplotlib.pyplot as plt
from src.tools import *

width, height= pyautogui.size()

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
try:
    while(True): 
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            x_max, x_min, y_max, y_min = get_bound(preds)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color=(245, 245, 245), thickness=2) 
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

In [94]:
# cv2.startWindowThread()
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
image = np.zeros((height, width, 3), np.uint8)
image[:,:,:] = 215
x, y = np.random.rand(), np.random.rand()
cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
cv2.imshow("window", image)
cv2.waitKey(2000)
cv2.destroyAllWindows()

# Collect

## Method 1

In [ ]:
X, Y = [], []

In [4]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
try:
    for i in range(100): 
        image = np.zeros((height, width, 3), np.uint8)
        image[:,:,:] = 215
        x, y = np.random.uniform(), np.random.uniform()
        Y.append([x,y])
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)
        cv2.waitKey(500)
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            box = get_box(get_bound(preds))
        except:
            pass
        
#         cv2.imshow('preview',warpBox(frame, box, target_height=32, target_width=64))
        X.append(warpBox(frame, box, target_height=32, target_width=64))
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
finally:
    cv2.destroyAllWindows()
    cap.release()

## Method 2

In [83]:
X, Y = [], []

In [85]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass

target_height, target_width = 32, 64

x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(3):
    path.append([np.random.uniform(), np.random.uniform()])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 1000
        n = 20
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            Y.append([xc,yc])
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=30, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
#             cv2.waitKey(T//n)
            ret, frame = cap.read()
            preds = landmarks_detector.get_landmarks(frame)
            try:
                box = get_box(get_bound(preds))
            except:
                pass
            X.append(warpBox(frame, box, target_height=target_height, target_width=target_width))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [86]:
X = np.array(X)/255.0
Y = np.array(Y[:len(X)])

print(X.shape, Y.shape)

(140, 64, 128, 3) (140, 2)


# Train model

In [87]:
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(target_height, target_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='adam', loss="mse")
model.summary() 

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 62, 126, 32)       896       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 63, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 29, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 30, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 12, 28, 64)        36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 21504)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)               

In [88]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(X))), train_size=0.9, shuffle=True)

/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [89]:
history = model.fit(X[train_set], Y[train_set], epochs=100, validation_data=(X[test_set], Y[test_set]))

Train on 126 samples, validate on 14 samples
Epoch 1/100
126/126 [==============================] - 1s 8ms/sample - loss: 0.1168 - val_loss: 0.1170
Epoch 2/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.1110 - val_loss: 0.1271
Epoch 3/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.1092 - val_loss: 0.1180
Epoch 4/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.1057 - val_loss: 0.1221
Epoch 5/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.1003 - val_loss: 0.1063
Epoch 6/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.0905 - val_loss: 0.0985
Epoch 7/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.0764 - val_loss: 0.0811
Epoch 8/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.0656 - val_loss: 0.0598
Epoch 9/100
126/126 [==============================] - 0s 4ms/sample - loss: 0.0535 - val_loss: 0.0617
Epoch 10/100
126/126 [======

126/126 [==============================] - 1s 4ms/sample - loss: 0.0036 - val_loss: 0.0035
Epoch 80/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0040 - val_loss: 0.0047
Epoch 81/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0036 - val_loss: 0.0060
Epoch 82/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0037 - val_loss: 0.0035
Epoch 83/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0033 - val_loss: 0.0038
Epoch 84/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0032 - val_loss: 0.0035
Epoch 85/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0031 - val_loss: 0.0036
Epoch 86/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0031 - val_loss: 0.0040
Epoch 87/100
126/126 [==============================] - 1s 4ms/sample - loss: 0.0032 - val_loss: 0.0033
Epoch 88/100
126/126 [==============================] - 1s 4ms/sample - loss:

# Test

In [90]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
image = np.zeros((height, width, 3), np.uint8)

try:
    while(True): 
        ret, frame = cap.read()
        preds = landmarks_detector.get_landmarks(frame)
        try:
            box = get_box(get_bound(preds))
        except:
            pass
        img = warpBox(frame, box, target_height=target_height, target_width=target_width) / 255.0
        [x, y] = model.predict(np.expand_dims(img, 0))[0]
        image[:,:,:] = 215
        cv2.circle(image, (int(x*width),int(y*height)), radius=30, color=(35,35,35), thickness=-2)
        cv2.imshow("window", image)        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise GetOutOfLoop
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()